https://github.com/sadeepj/crfasrnn_keras/blob/master/crfrnn_model.py

In [53]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, \
    Dropout, Conv2DTranspose, Cropping2D, Add

In [54]:
channels, height, weight = 3, 500, 500
# Input
input_shape = (height, weight, 3)
img_input = Input(shape=input_shape)

# Add plenty of zero padding
x = ZeroPadding2D(padding=(100, 100))(img_input)

print x
# VGG-16 convolution block 1
x = Conv2D(64, (3, 3), activation='relu', padding='valid', name='conv1_1')(x)
print x
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_2')(x)
print x
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)
print x


Tensor("zero_padding2d_10/Pad:0", shape=(?, 700, 700, 3), dtype=float32)
Tensor("conv1_1_9/Relu:0", shape=(?, 698, 698, 64), dtype=float32)
Tensor("conv1_2_9/Relu:0", shape=(?, 698, 698, 64), dtype=float32)
Tensor("pool1_9/MaxPool:0", shape=(?, 349, 349, 64), dtype=float32)


In [55]:
# VGG-16 convolution block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_1')(x)
print x
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_2')(x)
print x
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool2', padding='same')(x)
print x

Tensor("conv2_1_7/Relu:0", shape=(?, 349, 349, 128), dtype=float32)
Tensor("conv2_2_7/Relu:0", shape=(?, 349, 349, 128), dtype=float32)
Tensor("pool2_7/MaxPool:0", shape=(?, 175, 175, 128), dtype=float32)


In [56]:
# VGG-16 convolution block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_1')(x)
print x
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_2')(x)
print x
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_3')(x)
print x
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool3', padding='same')(x)
print x
pool3 = x

Tensor("conv3_1_7/Relu:0", shape=(?, 175, 175, 256), dtype=float32)
Tensor("conv3_2_7/Relu:0", shape=(?, 175, 175, 256), dtype=float32)
Tensor("conv3_3_7/Relu:0", shape=(?, 175, 175, 256), dtype=float32)
Tensor("pool3_7/MaxPool:0", shape=(?, 88, 88, 256), dtype=float32)


In [57]:
# VGG-16 convolution block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_1')(x)
print x
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_2')(x)
print x
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_3')(x)
print x
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool4', padding='same')(x)
print x
pool4 = x


Tensor("conv4_1_7/Relu:0", shape=(?, 88, 88, 512), dtype=float32)
Tensor("conv4_2_7/Relu:0", shape=(?, 88, 88, 512), dtype=float32)
Tensor("conv4_3_7/Relu:0", shape=(?, 88, 88, 512), dtype=float32)
Tensor("pool4_7/MaxPool:0", shape=(?, 44, 44, 512), dtype=float32)


In [58]:
# VGG-16 convolution block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_1')(x)
print x
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_2')(x)
print x
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_3')(x)
print x
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool5', padding='same')(x)
print x

Tensor("conv5_1_5/Relu:0", shape=(?, 44, 44, 512), dtype=float32)
Tensor("conv5_2_5/Relu:0", shape=(?, 44, 44, 512), dtype=float32)
Tensor("conv5_3_5/Relu:0", shape=(?, 44, 44, 512), dtype=float32)
Tensor("pool5_5/MaxPool:0", shape=(?, 22, 22, 512), dtype=float32)


In [59]:
x = Conv2D(4096, (7, 7), activation='relu', padding='valid', name='fc6')(x)
print x
x = Dropout(0.5)(x)
print x
x = Conv2D(4096, (1, 1), activation='relu', padding='valid', name='fc7')(x)
print x
x = Dropout(0.5)(x)
x = Conv2D(21, (1, 1), padding='valid', name='score-fr')(x)
print x

Tensor("fc6_6/Relu:0", shape=(?, 16, 16, 4096), dtype=float32)
Tensor("dropout_13/cond/Merge:0", shape=(?, 16, 16, 4096), dtype=float32)
Tensor("fc7_6/Relu:0", shape=(?, 16, 16, 4096), dtype=float32)
Tensor("score-fr_6/BiasAdd:0", shape=(?, 16, 16, 21), dtype=float32)


In [60]:
score2 = Conv2DTranspose(21, (4, 4), strides=2, name='score2')(x)
print score2

Tensor("score2_7/BiasAdd:0", shape=(?, ?, ?, 21), dtype=float32)


In [61]:
print pool4
score_pool4 = Conv2D(21, (1, 1), name='score-pool4')(pool4)
print score_pool4
score_pool4c = Cropping2D((5, 5))(score_pool4)
print score_pool4c
score_fused = Add()([score2, score_pool4c])
print score_fused

score4 = Conv2DTranspose(21, (4, 4), strides=2, name='score4', use_bias=False)(score_fused)
print score4

Tensor("pool4_7/MaxPool:0", shape=(?, 44, 44, 512), dtype=float32)
Tensor("score-pool4_8/BiasAdd:0", shape=(?, 44, 44, 21), dtype=float32)
Tensor("cropping2d_9/strided_slice:0", shape=(?, 34, 34, 21), dtype=float32)
Tensor("add_9/add:0", shape=(?, 34, 34, 21), dtype=float32)
Tensor("score4_2/conv2d_transpose:0", shape=(?, ?, ?, ?), dtype=float32)


In [67]:
# Skip connections from pool3
print pool3
score_pool3 = Conv2D(21, (1, 1), name='score-pool3')(pool3)
print score_pool3
score_pool3c = Cropping2D((9, 9))(score_pool3)
print score_pool3c

# Fuse things together
score_final = Add()([score4, score_pool3c])
print score_final

# Final up-sampling and cropping
upsample = Conv2DTranspose(21, (16, 16), strides=8, name='upsample', use_bias=False)(score_final)
print upsample
upscore = Cropping2D(((31, 37), (31, 37)))(upsample)
print upscore

Tensor("pool3_7/MaxPool:0", shape=(?, 88, 88, 256), dtype=float32)
Tensor("score-pool3_4/BiasAdd:0", shape=(?, 88, 88, 21), dtype=float32)
Tensor("cropping2d_19/strided_slice:0", shape=(?, 70, 70, 21), dtype=float32)
Tensor("add_15/add:0", shape=(?, 70, 70, 21), dtype=float32)
Tensor("upsample_4/conv2d_transpose:0", shape=(?, ?, ?, ?), dtype=float32)
Tensor("cropping2d_20/strided_slice:0", shape=(?, ?, ?, ?), dtype=float32)


In [71]:
upscore2 = Add()([upscore,upsample])

ValueError: Operands could not be broadcast together with shapes (500, 500, 21) (568, 568, 21)

In [70]:
upscore2

<tf.Tensor 'add_16/add:0' shape=(?, ?, ?, ?) dtype=float32>